# Read data from densification test

## Imports

In [2]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FormatStrFormatter

## Códigos

In [3]:
def read_info(info_path):
    with open(info_path) as file:
        info = json.load(file)
    return info
    
def write_info(info_path, info):
    json_object = json.dumps(info, indent = 2)
    with open(info_path, 'w') as file:
        file.write(json_object)
        file.close()

def get_data(folder, exp_setting, res_dict, check):
    res_dict[exp_setting] = {}
    info = read_info(os.path.join(folder, 'metrics.json'))

    res_dict[exp_setting]['Tempo'] = round(info['tempo_train'], 5)
    keys_in = ['num_gaussians', 'PSNR', 'SSIM', 'LPIPS', 'FPS']
    keys_out = ['Number Gaussians', 'PSNR', 'SSIM', 'LPIPS', 'FPS']
    for key_in, key_out in zip(keys_in, keys_out):
        res_dict[exp_setting][key_out] = round(info[str(check)][key_in], 5)
    return res_dict

def get_df(json_folder, es=True, title='', check='10000'):
    res_dict = {}
    
    for folder in sorted(os.listdir(json_folder), key=lambda a: int(a.split()[3].removeprefix('refine-every_')), reverse=False):
        if folder.split()[1].endswith(str(es)):
            res_dict = get_data(os.path.join(json_folder, folder), folder, res_dict, check)

    df = pd.DataFrame(res_dict).T
    if len(res_dict.keys()) > 0:
        plot_up(df.index.tolist(), df, "PSNR", "Number Gaussians", title)
        plot_down(df.index.tolist(), df, "SSIM", "LPIPS", title)
    
    return df

def plot_up(xs, df, key1, key2, title):
    var1 = df[key1]
    var2 = df[key2]

    xs=[*map(lambda t:t.split()[2].replace('reset-alpha-every', 'rae') + ' ' + t.split()[3].replace('refine-every', 're'), xs)]
    
    x = np.arange(len(xs))      # numeric x locations
    width = 0.4                     # width of the bars

    fig, ax1 = plt.subplots(figsize=(10, 2))
    
    # First axis (left y-axis)
    bars1 = ax1.bar(x - width/2, var1, width, color="tab:blue", label="Variable 1")
    ax1.set_ylabel(key1, color="tab:blue")
    ax1.yaxis.set_major_formatter(FormatStrFormatter('%02.2f'))
    ax1.tick_params(axis="y", labelcolor="tab:blue")
    ma = max(var1)
    mi = min(var1)
    ax1.set_ylim(mi - abs(ma - mi) * 0.1, ma + abs(ma - mi) * 0.1)
    
    # Second axis (right y-axis)
    ax2 = ax1.twinx()
    bars2 = ax2.bar(x + width/2, var2, width, color="tab:orange", label="Variable 2")
    ax2.set_ylabel(key2, color="tab:orange")
    ax2.yaxis.set_major_formatter(FormatStrFormatter('%06.1f'))
    ax2.tick_params(axis="y", labelcolor="tab:orange")
    ma = max(var2)
    mi = min(var2)
    ax2.set_ylim(mi - abs(ma - mi) * 0.1, ma + abs(ma - mi) * 0.1)
    
    # X-axis settings
    # ax1.set_xticks(x)
    # ax1.set_xticklabels(xs, rotation=90)
    # ax1.set_xlabel("Categories")
    ax1.set_xticks([])
    
    plt.grid(True)
    plt.title(title)
    plt.show()

def plot_down(xs, df, key1, key2, title):
    var1 = df[key1]
    var2 = df[key2]

    xs=[*map(lambda t:t.split()[2].replace('reset-alpha-every', 'rae') + ' ' + t.split()[3].replace('refine-every', 're'), xs)]
    
    x = np.arange(len(xs))      # numeric x locations
    width = 0.4                     # width of the bars

    fig, ax1 = plt.subplots(figsize=(10, 2))
    
    # First axis (left y-axis)
    bars1 = ax1.bar(x - width/2, var1, width, color="tab:green", label="Variable 1")
    ax1.set_ylabel(key1, color="tab:green")
    ax1.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    ax1.tick_params(axis="y", labelcolor="tab:green")
    ma = max(var1)
    mi = min(var1)
    ax1.set_ylim(mi - abs(ma - mi) * 0.1, ma + abs(ma - mi) * 0.1)
    
    # Second axis (right y-axis)
    ax2 = ax1.twinx()
    bars2 = ax2.bar(x + width/2, var2, width, color="tab:red", label="Variable 2")
    ax2.set_ylabel(key2, color="tab:red")
    ax2.yaxis.set_major_formatter(FormatStrFormatter('%.4f'))
    ax2.tick_params(axis="y", labelcolor="tab:red")
    ma = max(var2)
    mi = min(var2)
    ax2.set_ylim(mi - abs(ma - mi) * 0.1, ma + abs(ma - mi) * 0.1)
    
    # X-axis settings
    ax1.set_xticks(x)
    ax1.set_xticklabels(xs, rotation=90)
    ax1.set_xlabel("Testes")

    plt.grid(True)
    # plt.title(title)
    plt.show()

## Casa

In [ ]:
dataset = 'casa'
check = '30000'

#### Splatfacto-big | Bilateral mudado and Scale ambos False

In [ ]:
json_folder = f'/home/tafnes/Documentos/teste_casa/{dataset}_jsons/splatfacto-big bilateral_False'

get_df(json_folder, es=False, title='Splatfacto big', check=check)

#### Splatfacto-big | Bilateral mudado and Scale ambos True

In [ ]:
json_folder = f'/home/tafnes/Documentos/teste_casa/{dataset}_jsons/splatfacto-big bilateral_True'

get_df(json_folder, es=False, title='Splatfacto big com bilateral mudado e scale reg', check=check)

#### Splatfacto-mcmc | Bilateral mudado and Scale ambos False

In [ ]:
json_folder = f'/home/tafnes/Documentos/teste_casa/{dataset}_jsons/splatfacto-mcmc bilateral_False'

get_df(json_folder, es=False, title='Splatfacto mcmc', check=check)

## Truck

In [ ]:
dataset = 'truck'
check = '30000'

#### Splatfacto-big | Bilateral mudado and Scale ambos False

In [ ]:
json_folder = f'/home/tafnes/Documentos/teste_casa/{dataset}_jsons/splatfacto-big bilateral_False'

get_df(json_folder, es=False, title='Splatfacto big', check=check)

#### Splatfacto-big | Bilateral mudado and Scale ambos True

In [ ]:
json_folder = f'/home/tafnes/Documentos/teste_casa/{dataset}_jsons/splatfacto-big bilateral_True'

get_df(json_folder, es=False, title='Splatfacto big com bilateral mudado e scale reg', check=check)

#### Splatfacto-mcmc | Bilateral mudado and Scale ambos False

In [ ]:
json_folder = f'/home/tafnes/Documentos/teste_casa/{dataset}_jsons/splatfacto-mcmc bilateral_False'

get_df(json_folder, es=False, title='Splatfacto mcmc', check=check)

## Train

In [ ]:
dataset = 'train'
check = '30000'

#### Splatfacto-big | Bilateral mudado and Scale ambos False

In [ ]:
json_folder = f'/home/tafnes/Documentos/teste_casa/{dataset}_jsons/splatfacto-big bilateral_False'

get_df(json_folder, es=False, title='Splatfacto big', check=check)

#### Splatfacto-big | Bilateral mudado and Scale ambos True

In [ ]:
json_folder = f'/home/tafnes/Documentos/teste_casa/{dataset}_jsons/splatfacto-big bilateral_True'

get_df(json_folder, es=False, title='Splatfacto big com bilateral mudado e scale reg', check=check)

#### Splatfacto-mcmc | Bilateral mudado and Scale ambos False

In [ ]:
json_folder = f'/home/tafnes/Documentos/teste_casa/{dataset}_jsons/splatfacto-mcmc bilateral_False'

get_df(json_folder, es=False, title='Splatfacto mcmc', check=check)